In [ ]:
from src.utils import create_square, plot_function
from src.models.cell_models import ReparametrizedFitzHughNagumo
from src.models.dynamics_models import BidomainModel
import numpy as np
import ufl

In [ ]:
domain = create_square(32, 32)
x = ufl.SpatialCoordinate(domain)
cell_model = ReparametrizedFitzHughNagumo()


class Model(BidomainModel):
    def __init__(
        self,
    ):
        pass

    def initial_V_m(self):
        return lambda x: x[0] ** 2 + x[1] ** 2 < 0.2**2, 0.0

    def ischemia(self):
        return (
            (x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2 < 0.5**2,
            model.M_i / 10,
            model.M_e / 10,
        )

model = Model()
model.setup(domain)

### Solve the equations with given parameters

In [ ]:
V, signal = model.solve(
    T=500,
    steps=500,
    cell_model=cell_model,
    gif_name="demo.gif",
    signal_point=[0.2, 0.2, 0.0],
)

### Plotting a final plot of transmembrane potential

In [ ]:
plot_function(V)

### Plotting a transmembrane potential in a given signal point

In [ ]:
import matplotlib.pyplot as plt

time = np.linspace(0, 200, len(signal))
plt.plot(time, signal)
plt.xlabel("time [ms]")
plt.ylabel("signal [mV]")
plt.title("Time dependence of transmembrane potential")

### Plotting a transmembrane potential solution of a cell model

In [ ]:
cell_model.visualize(300, -60, 0)